In [51]:
import json
from glob import glob

In [52]:
# CONFIG = 'hopper_custom_rl' 
# DATASET = 'H-MIXED-RT-4'
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Hopper_H-MIXED-RT-4/*/*"
# rollout_lengths = {1, 5, 50}

CONFIG = 'halfcheetah_custom_rl' 
DATASET = 'MIXED-RT-5'
model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/*/*"
rollout_lengths = {1, 5, 20}

# CONFIG = 'walker2d_custom_rl' 
# DATASET = 'W-MIXED-RT-1' 
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Walker2d_W-MIXED-RT-1/*/*"
# rollout_lengths = {1, 5, 10}

mopo_penalty_coeffs = {0, 1, 5}
seeds = {1, 2, 3}
# seeds = {1,}

In [53]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s

['/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexb20/s1_2023-03-06_11-15-54u_uink5n',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexstdb01/s1_2023-03-06_11-18-10v4rza8b0',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexstdb8/s1_2023-03-06_11-33-50_f7wccrn',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexstdb1/s1_2023-03-06_11-20-08ku3tcgc6',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexb10/s1_2023-03-06_11-15-54qsw2r2kx',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexstdb05/s1_2023-03-06_11-18-12zi2l3jek',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/rexb5/s1_2023-03-06_19-33-0071jd29j5',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-5/default/s1_2023-03-06_11-13-45xh8hsuja',
 '/rds/project/rds-eWkDxBhxBrQ

In [54]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s = ["/".join(model_dir.split('/')[-3:])for model_dir in with_s]
model_load_dirs = with_s
with_s

['HalfCheetah_MIXED-RT-5/rexb20/s1_2023-03-06_11-15-54u_uink5n',
 'HalfCheetah_MIXED-RT-5/rexstdb01/s1_2023-03-06_11-18-10v4rza8b0',
 'HalfCheetah_MIXED-RT-5/rexstdb8/s1_2023-03-06_11-33-50_f7wccrn',
 'HalfCheetah_MIXED-RT-5/rexstdb1/s1_2023-03-06_11-20-08ku3tcgc6',
 'HalfCheetah_MIXED-RT-5/rexb10/s1_2023-03-06_11-15-54qsw2r2kx',
 'HalfCheetah_MIXED-RT-5/rexstdb05/s1_2023-03-06_11-18-12zi2l3jek',
 'HalfCheetah_MIXED-RT-5/rexb5/s1_2023-03-06_19-33-0071jd29j5',
 'HalfCheetah_MIXED-RT-5/default/s1_2023-03-06_11-13-45xh8hsuja',
 'HalfCheetah_MIXED-RT-5/rexstdb2/s1_2023-03-06_11-20-08c41y0uzr']

In [55]:
# model_load_dirs = ['HalfCheetah_MIXED-RT-5/default/s1_2023-03-06_11-13-45xh8hsuja']

In [57]:
list_to_dump = []

for mld in model_load_dirs:
    bnn_name = mld.split('/')[-2]
    bnn_seed = mld.split('/')[-1].split('_')[0]
    for rl in rollout_lengths:
        for mp in mopo_penalty_coeffs:
            for seed in seeds:
                exp_name = f'{bnn_name}{bnn_seed}_mp{mp}rl{rl}'
                list_to_dump.append(dict(config=CONFIG, dataset=DATASET, rollout_length=rl, mopo_penalty_coeff=mp,
                                    seed=seed, model_load_dir=mld, exp_name=exp_name)) 

with open(f"{DATASET}_exp_params.json", "w") as outfile:
    json.dump(list_to_dump, outfile, indent="")

In [58]:
f"{DATASET}_exp_params.json"

'MIXED-RT-5_exp_params.json'